In [59]:
import pandas as pd

In [60]:
df = pd.read_csv('raw_data.csv')

In [61]:
dfc = pd.read_csv('cleaned-data.csv')

In [51]:
print(df.price.isna().sum())
print(dfc.price.isna().sum())

dfc = df.dropna(subset=['price'], how='all')
dfc.reset_index(inplace=True)

36
0


In [ ]:
dfc.drop(['index'], axis=1, inplace=True)
print(dfc)
#

In [58]:
dfc.to_csv('cleaned-data.csv', index=False)